In [ ]:
from google.colab import drive
drive.mount('/content/drive')
ROOT = '''/content/drive/MyDrive/Colab Notebooks/googledrive/wordAssociation'''

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import csv
import json
import sqlite3
import numpy as np
from copy import deepcopy
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from datetime import datetime

In [ ]:
def plot_learning_curve_single_subject(marks_conf,counterbalance):

    fig, ax = plt.subplots(figsize=(5, 3))
    rect = patches.Rectangle((0,-0.2), 3, 1.4, linewidth=1,  facecolor='yellow', alpha=0.5)#edgecolor='r',
    conf_mat = marks_conf[9]
    marks_acc = [(conf_mat.diagonal() / (conf_mat.sum(1)+1e-7)).mean()]
    for i in range(1,6):
        conf_mat = marks_conf[(i+1)*8-1] - marks_conf[i*8-1]
        acc = (conf_mat.diagonal() / (conf_mat.sum(1)+1e-7)).mean()
        marks_acc.append(acc)

    ax.scatter(range(len(marks_acc)),marks_acc,c='black',s=60)

    for i in range(len(marks_acc)-1):
        ax.plot([i,i+1],[marks_acc[i],marks_acc[i+1]],c='black')
    plt.yticks([_*0.1 for _ in range(0,11,2)],[round(_*0.1,1) for _ in range(0,11,2)],fontsize=20)
    plt.xticks(range(len(marks_acc)),[_*8 for _ in range(1,len(marks_acc)+1)],fontsize=20)
    plt.suptitle('counterbalance: {}'.format(counterbalance), fontsize=20)
    # Add the patch to the Axes
    ax.add_patch(rect)
    ax.xaxis.set_ticks_position('none')
    ax.yaxis.set_ticks_position('none')
    ax.spines[['right', 'left']].set_visible(False)

def construct_confmat():
    labels = ['h_word2vec','h_gpt2','h_gpt3emb','h_gpt3promptcurie','h_gpt3promptdav']#,'h_catch']
    confmatrices = {**{'h_machine':np.zeros((2,2))}, **{label: np.zeros((1,2)) for label in labels}}
    return confmatrices

def print_metadata(data):
    print("====="*20)
    datetime_object1 = datetime.strptime(data[11].split('.')[0], '%Y-%m-%d %H:%M:%S')
    datetime_object2 = datetime.strptime(data[13].split('.')[0], '%Y-%m-%d %H:%M:%S')
    time_used = datetime_object2 - datetime_object1
    time_used = round(time_used.seconds/60,1)
    print('[Worker ID]',json.loads(data[-1])['workerId'])
    print("[Finished Date]",datetime_object2)
    print("[Time Used] {} mins".format(time_used))
    print("[Questionnaire]", json.loads(data[-1])["questiondata"])
    print("[Counterbalance]", json.loads(data[-1])["counterbalance"])
    return time_used, json.loads(data[-1])["counterbalance"]

def get_gt_from_pagename(pagename):
    # example:  'page_name': '568_e_0.html/',
    return {'a':'Human','b':'Machine','c':'Machine','d':'Machine','e':'Machine','f':'Machine','g':'Catch'}[pagename.split('_')[2]]

def get_model_from_pagename(pagename):
    # example:  'page_name': '568_e_0.html/',
    return {'b':'word2vec','c':'gpt2','d':'gpt3emb','e':'gpt3promptcurie','f':'gpt3promptdav'}[pagename.split('_')[2]]

def isRandomStringCatch(pagename):
    with open(ROOT+"/Feedback_Word/static/responses_wordassoc/"+pagename.split('/')[-1],'r') as f:
        line = f.readlines()[0]

    cue = line.split('; Association')[0].split('Cue: ')[-1].strip()
    assoc = line.split('Association: ')[-1].strip().replace(' </p>','')
    return cue != assoc
#     print(words)

conn = sqlite3.connect(ROOT+"/Feedback_Word/ec2_db/turing_feedback_word.db")
cursor = conn.cursor()
cursor.execute("select * from TuringAssociationAI_turing;")

label_dict = {'human':0,'machine':1,'Human':0,'Machine':1,'word2vec':1,'gpt2':1,'gpt3emb':1,'gpt3promptcurie':1,'gpt3promptdav':1}
oneWord_curr_d = {'0':True,'1':False} # dummy test gt
topic_d = {'1':True,'2':False} # user response to dummy tests
modelname_dict = {'word2vec':'Word2Vec','gpt2':'GPT2','gpt3emb':'GPT3(Emb)','gpt3promptcurie':'GPT3(pr, Curie)','gpt3promptdav':'GPT3(pr, DaVinci)'}

count = 0
participant_count = 0
trials_dict = {}
all_data = cursor.fetchall()
completed_counterbalances = []


confmat_overall = construct_confmat()
catch_trial_mat_overall = np.zeros((1,2))
swap_trial_mat_overall = np.zeros((1,2))
for i in range(len(all_data)):
    if all_data[i][15] in [5]:
#         print(all_data[i][11])
        data_dict = json.loads(all_data[i][17])
        if data_dict['mode'] != 'live':
            continue
        time_used, counterbalance = print_metadata(all_data[i])
        completed_counterbalances.append(counterbalance)
        marks = []
        marks_conf = []
        confmatrices = construct_confmat() # without catch
        catch_trial_mat = np.zeros((1,2))
        swap_trial_mat = np.zeros((1,2))
#         confmatrices_without_catch = construct_confmat()
#     if True:

        participant_count += 1
        nTrial = 0
        for dat in data_dict['data']:
            if dat['trialdata']['phase'] == 'TEST':
                nTrial+=1
                trial_data = {}
                trial_data['workerID'] = data_dict['workerId']
                trial_data['workerData'] = data_dict['questiondata']
                trial_data['trialData'] = dat['trialdata']


#                 trials_dict['Trial_'+str(count)] = trial_data
#                 trial_data = trial_data['trialData']
                trial_data['trialData']['correct response'] = get_gt_from_pagename(trial_data['trialData']['page_name'])
                trial_data['trialData']['choose response'] = trial_data['trialData']['Aclass']

#                 print(trial_data)
                if trial_data['trialData']['correct response'] == 'Catch':
                    if not isRandomStringCatch(trial_data['trialData']['page_name']):
                        catch_trial_mat[0][label_dict[trial_data['trialData']['choose response']]] += 1
                        catch_trial_mat_overall[0][label_dict[trial_data['trialData']['choose response']]] += 1


                elif trial_data['trialData']['correct response'] == 'Swapped':
                    swap_trial_mat[0][label_dict[trial_data['trialData']['choose response']]] += 1
                    swap_trial_mat_overall[0][label_dict[trial_data['trialData']['choose response']]] += 1
                elif nTrial > 40:
                    confmatrices['h_machine'][label_dict[trial_data['trialData']['correct response']]][label_dict[trial_data['trialData']['choose response']]] += 1
                    confmat_overall['h_machine'][label_dict[trial_data['trialData']['correct response']]][label_dict[trial_data['trialData']['choose response']]] += 1
                    if trial_data['trialData']['correct response'] == "Machine":
                        model = get_model_from_pagename(trial_data['trialData']['page_name'])
                        confmatrices['h_{}'.format(model)][0][label_dict[trial_data['trialData']['choose response']]] += 1
                        confmat_overall['h_{}'.format(model)][0][label_dict[trial_data['trialData']['choose response']]] += 1
                    marks.append(1 if trial_data['trialData']['correct response'] == trial_data['trialData']['choose response'] else 0)
                    marks_conf.append(deepcopy(confmatrices['h_machine']))

#                 trial_data['trialData'] = dat['trialdata']
                if trial_data['trialData']['correct response']  in ['Catch','Swapped'] or nTrial <=40:
                    continue

                count += 1
                trial_data['trialData']['response_gender'] = trial_data['trialData']['Agender']
                del trial_data['trialData']['Agender']

                trial_data['trialData']['response_speaker'] = trial_data['trialData']['choose response'].lower()
                del trial_data['trialData']['choose response']

#                 trial_data['trialData']['response_object'] = trial_data['trialData']['topic'].lower()
#                 del trial_data['trialData']['topic']

                trial_data['trialData']['groundtruth'] = trial_data['trialData']['correct response'].lower()


#                 trial_data['trialData']['object_groundtruth'] = trial_data['trialData']['oneWord_curr'].lower()
#                 del trial_data['trialData']['oneWord_curr']

                if trial_data['trialData']['correct response'] in ["Machine","Catch","Swapped"]:
                    trial_data['trialData']['machine_groundtruth'] = modelname_dict[model.lower()]
                else:
                    trial_data['trialData']['machine_groundtruth'] = ''

                del trial_data['trialData']['correct response']
#                 trial_data['trialData']["imgtype"] = "naturaldesign"



                trials_dict['Trial_Word_'+str(count)] = trial_data
#         plot_learning_curve_single_subject(marks_conf,participant_count)
        print(confmatrices)
        print('catch',catch_trial_mat)
#         print('swap',swap_trial_mat)
#     else:
#         print(all_data[i][15])
conn.close()

[Worker ID] A38DC3BG1ZCVZ2
[Finished Date] 2025-04-03 07:53:39
[Time Used] 7.2 mins
[Questionnaire] {'country': 'United States', 'age': '36-40', 'engagement': '8', 'difficulty': '9', 'mode': 'live', 'gender': 'female', 'education': 'bachelor', 'ai-experience': '2', 'condition': 0, 'native': 'yes'}
[Counterbalance] 0
{'h_machine': array([[14.,  6.],
       [10., 10.]]), 'h_word2vec': array([[0., 4.]]), 'h_gpt2': array([[2., 2.]]), 'h_gpt3emb': array([[4., 0.]]), 'h_gpt3promptcurie': array([[2., 2.]]), 'h_gpt3promptdav': array([[2., 2.]])}
catch [[0. 6.]]
[Worker ID] A3UBB1LJKDQGZP
[Finished Date] 2025-04-03 07:54:56
[Time Used] 7.3 mins
[Questionnaire] {'country': 'United States', 'age': '41-45', 'engagement': '10', 'difficulty': '3', 'mode': 'live', 'gender': 'female', 'education': 'bachelor', 'ai-experience': '2', 'condition': 0, 'native': 'yes'}
[Counterbalance] 0
{'h_machine': array([[13.,  7.],
       [13.,  7.]]), 'h_word2vec': array([[2., 2.]]), 'h_gpt2': array([[2., 2.]]), 'h_gp

In [ ]:
catch_trial_mat_overall

array([[102., 398.]])

In [ ]:
catch_trial_mat_overall

array([[ 91., 266.]])

In [ ]:
266/(91+266)

0.7450980392156863

In [ ]:
confmat_overall

{'h_machine': array([[1275.,  725.],
        [1080.,  920.]]),
 'h_word2vec': array([[128., 272.]]),
 'h_gpt2': array([[226., 174.]]),
 'h_gpt3emb': array([[241., 159.]]),
 'h_gpt3promptcurie': array([[256., 144.]]),
 'h_gpt3promptdav': array([[229., 171.]])}

In [ ]:
with open("/home/liuxiao/TuringGithubRunnable/wordAssociation/Plot/Data_all/humanjudge_Word_Feedback_testphase.json", 'w') as f:
    json.dump(trials_dict,f)

In [ ]:
1267/2000

0.6335

Bad pipe message: %s [b'\x9f\xab\xfe \x03\xa9\x89h\xc6\xa4\xdb\xcb\xe7+F,\xb7\x89\x00\x02\xbc\x00\x00\x00\x01\x00\x02\x00\x03\x00\x04\x00\x05\x00\x06\x00\x07\x00\x08\x00\t\x00\n\x00\x0b\x00\x0c\x00\r\x00\x0e\x00\x0f\x00\x10\x00\x11\x00\x12\x00\x13\x00\x14\x00\x15\x00\x16\x00\x17\x00\x18\x00\x19\x00\x1a\x00\x1b\x00\x1e\x00\x1f\x00 \x00!\x00"\x00#\x00$\x00%\x00&\x00\'\x00(\x00)\x00*\x00+\x00,\x00-\x00.\x00/\x000\x001\x002\x003\x004\x005\x006\x007\x008\x009\x00:\x00;\x00<\x00=\x00>\x00?\x00@\x00', b'B\x00C\x00D\x00E\x00F\x00g\x00h\x00i\x00j\x00k\x00l\x00m\x00\x84\x00\x85\x00\x86\x00\x87\x00\x88\x00\x89\x00\x8a\x00\x8b\x00\x8c\x00\x8d\x00\x8e\x00\x8f\x00\x90\x00\x91\x00\x92\x00\x93\x00\x94\x00\x95\x00\x96\x00\x97\x00']
Bad pipe message: %s [b'\x99\x00\x9a\x00\x9b\x00\x9c\x00\x9d\x00\x9e\x00\x9f\x00\xa0\x00\xa1\x00\xa2\x00\xa3\x00\xa4\x00\xa5\x00\xa6\x00\xa7\x00\xa8\x00\xa9\x00\xaa\x00\xab\x00\xac\x00\xad\x00\xae\x00\xaf\x00\xb0\x00\xb1\x00\xb2\x00\xb3\x00\xb4\x00\xb5\x00\xb6\x00\xb7\x00\xb